# Convolutional Autoencoders
Usually when dealing with images, the encoder and decoder parts of an autoencoder are typically made of convolutional neural network layers.   This workbook will look at this more common autoencoder.

One change when we read the data in: we need to shape it appropriately as a 28x28x1 image.

Another change: since the CNN version takes **much** longer to train, we will do the following:
1.  We train a short version in this workbook.   After training, we will save all of the relevant info (the models and the history information returned from training) to files.
2.  We have a longer version that is almost exactly the same code, but running over all of the data.  The longer version is in python, and we have a pbs shell script you can submit to run on the batch system.  As above, after training, we will save all of the relevant info (the models and the history information returned from training) to files (named differently of course).
3.  We will split the analysis part (looking at plots and so on) off into another workbook.  In this other workbook, you can use the files from this workbook, or the files that are made from the pbs/python version.  

**Before** starting this workbook, open up a Pitzer shell, navigate to this directory, and type at the prompt:
    
    qsub pbs_run_ae_cnn.sh
    
This submits the longer python version of the code to the batch system.   Hopefully it will be close to finishing by the time you get to the end of this workbook.
    
After submitting the above script, you can go through this workbook.  Make sure you take some time to compare the python code run in the above script to the code in this workbook.

## Get the data

In [ ]:
from keras.datasets import mnist
import numpy as np
import scipy.io as sio
#
# See this for more info: https://arxiv.org/pdf/1702.05373.pdf
mat = sio.loadmat('/fs/scratch/PAS1043/physics6820/emnist/matlab/emnist-byclass.mat')
#print(mat)

data = mat['dataset']

ex_train = data['train'][0,0]['images'][0,0]
ey_train = data['train'][0,0]['labels'][0,0]
ex_test = data['test'][0,0]['images'][0,0]
ey_test = data['test'][0,0]['labels'][0,0]

ex_train = ex_train.reshape( (ex_train.shape[0], 28,28), order='F')
ex_test = ex_test.reshape( (ex_test.shape[0], 28,28), order='F')

ex_train = ex_train.reshape( (ex_train.shape[0], 784))
ex_test = ex_test.reshape( (ex_test.shape[0], 784))
ex_train = ex_train.astype('float32') / 255.
ex_test = ex_test.astype('float32') / 255.


import pandas as pd

df_train = pd.DataFrame(ex_train)
df_train['label'] = ey_train
df_digits_train = df_train[df_train['label']<=9]
x_train = df_digits_train.iloc[:50000,:784].values
x_train = x_train.reshape((x_train.shape[0],28,28,1))
y_train = df_digits_train.loc[:50000,'label'].values

df_test = pd.DataFrame(ex_test)
df_test['label'] = ey_test
df_digits_test = df_test[df_test['label']<=9]
x_test = df_digits_test.iloc[:,:784].values
x_test = x_test.reshape((x_test.shape[0],28,28,1))
y_test = df_digits_test['label'].values

#
from keras.utils import to_categorical

y_train_labels_cat = to_categorical(y_train)
y_test_labels_cat = to_categorical(y_test)


In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
for digit,count in zip(unique, counts):
    print("digit",digit,"; count ",count)

## Designing the autoencoder.   
Again we will use the keras [blog](https://blog.keras.io/building-autoencoders-in-keras.html) on autoencoders to guide us on the design.   However, we will use a syntax different from that source, and also explicitly design our auotoencoder model as made up of two **sub-models**.  Carefully examine the syntax below so you are sure it makes sense.

Why do we do this?   After training, we can of course save the **autoencoder** model using the keras **save** method.  However, we can **also** save the sub-models (which we naturally call **encoder** and **decoder**).   This will be very important when we examine the use of **stacked** autoencoders in classification.

In our CNN, we have:
1.  An input layer: This is just the 784 pixels from the image.
2.  The encoder: this layer has 784 inputs, and an output of dimension 4x4x8=128.  This is the same size as our encoder using the fully connected layers.  However, the encoder here uses 3 convolutional layers and 3 max-pooling layers.
3.  The decoder: this layer takes the 128 outputs of the encoder as input, then has 784 outputs.  You will notice that the decoder uses **updampling**: UpSampling2D is just a simple scaling up of the image by resizing upwards.   This decoder uses 3 convolutional layers and 3 upsampling layers.


In [ ]:
from keras import models
from keras import layers
from keras import regularizers

# make our encoder
encoder = models.Sequential()
#
# First convolutional layer
encoder.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same',input_shape=(28,28,1)))
encoder.add(layers.MaxPooling2D((2,2), padding='same'))
encoder.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
encoder.add(layers.MaxPooling2D((2,2), padding='same'))
encoder.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
encoder.add(layers.MaxPooling2D((2,2), padding='same'))
print("encoder===>")
print(encoder.summary())

#
# Now make the decoder
# make our encoder
decoder = models.Sequential()
#
# First convolutional layer
decoder.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same',input_shape=(4,4,8)))
decoder.add(layers.UpSampling2D((2,2)))
decoder.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
decoder.add(layers.UpSampling2D((2,2)))
decoder.add(layers.Conv2D(16, (3, 3), activation='relu'))
decoder.add(layers.UpSampling2D((2,2)))
decoder.add(layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same'))
print("decoder===>")
print(decoder.summary())

#

autoencoder = models.Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)
#
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy',metrics=['mse'])
#autoencoder.compile(optimizer='adadelta', loss='mse',metrics=['mse'])
print("autoencoder===>")
print(autoencoder.summary())

In [ ]:
history = autoencoder.fit(x_train, x_train,
                epochs=5,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
print(history.history)

## Saving our models

In [ ]:
encoder.save('fully_trained_encoder_cnn_small.h5')
decoder.save('fully_trained_decoder_cnn_small.h5')
autoencoder.save('fully_trained_autoencoder_cnn_small.h5')


## Saving our history data
It is helpful to save the history data (note we only save the "history.history" part - otherwise the resulting saved object is **very** large) for plotting performance results later.   To do this we use the python **pickle** module.

The pickle module implements an algorithm for serializing and de-serializing a Python object structure. “Pickling” is the process whereby a Python object hierarchy is converted into a byte stream, and “unpickling” is the inverse operation, whereby a byte stream is converted back into an object hierarchy.  Note: **don't** try to use pickle for keras models!   Use the **save** method as above.

In [ ]:
import pickle 
print(history.history)
pickle.dump(history.history,open('history_cnn_small.pkl', 'wb') )

## Now go the analysis workbook
In this directory, open up "ana_cnn.ipynb" and continue from there.